In [1]:

from __future__ import print_function

import sys

# sys.path.extend(['/home/jogi/git/repository/dcnn_acdc'])
sys.path.extend(['/home/jorg/repository/dcnn_acdc'])

import matplotlib
import matplotlib.pyplot as plt

from matplotlib import cm


from datetime import datetime
from pytz import timezone
import itertools
import copy

import SimpleITK as sitk
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init

import numpy as np
import dill
import os
import glob
from collections import OrderedDict

from in_out.load_data import ACDC2017DataSet
from config.config import config
from utils.experiment import Experiment, ExperimentHandler
from utils.batch_handlers import TwoDimBatchHandler
from utils.test_handler import ACDC2017TestHandler
from plotting.learning_curves import loss_plot
from plotting.uncertainty_plots import analyze_slices

from models.dilated_cnn import BaseDilated2DCNN

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [4]:
# Constants aka fixed variables
ROOT_DIR = "/home/jorg/repository/dcnn_acdc"
LOG_DIR = os.path.join(ROOT_DIR, "logs")
CYCLE_OFFSET = 100000
# set seeds
SEED = 4325
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.enabled = True
np.random.seed(SEED)

# MC dropout models
# Fold 2: 20180328_10_54_36_dcnn_mcv1_150000E_lr2e02
# Fold 0: 20180330_09_56_01_dcnn_mcv1_150000E_lr2e02
exp_mc005_model = os.path.join(LOG_DIR, "20180328_10_54_36_dcnn_mcv1_150000E_lr2e02")
# Fold 2: 20180329_08_04_12_dcnn_mcv1_150000E_lr2e04
# Fold 0: 20180331_09_49_14_dcnn_mcv1_150000E_lr2e02
exp_mc005_bs_model = os.path.join(LOG_DIR, "20180329_08_04_12_dcnn_mcv1_150000E_lr2e04")


# Fold 2: 20180328_10_54_55_dcnn_mcv1_150000E_lr2e02
# Fold 0: 20180330_09_56_15_dcnn_mcv1_150000E_lr2e02
exp_mc01_model = os.path.join(LOG_DIR, "20180328_10_54_55_dcnn_mcv1_150000E_lr2e02")
# Fold 2: 20180329_08_03_29_dcnn_mcv1_150000E_lr2e04
# Fold 0: 20180331_09_48_48_dcnn_mcv1_150000E_lr2e02
exp_mc01_bs_model = os.path.join(LOG_DIR, "20180329_08_03_29_dcnn_mcv1_150000E_lr2e04")

exper_mc005 = ExperimentHandler.load_experiment(exp_mc005_model)
training_fold = exper_mc005.run_args.fold_ids[0]
print("!!! IMPORTANT !!! - EVALUATING MODELS ON HOLD OUT IMAGES IN FOLD {}".format(training_fold))

exper_hdl_mc005 = ExperimentHandler(exper_mc005, use_logfile=False)
exper_hdl_mc005.set_root_dir(ROOT_DIR)
exper_hdl_mc005.set_model_name("MC-dropout fold-{} (p={})".format(exper_hdl_mc005.exper.run_args.fold_ids[0],
    exper_hdl_mc005.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc005.exper.model_name))

exper_mc005_bs = ExperimentHandler.load_experiment(exp_mc005_bs_model)
exper_hdl_mc005_bs = ExperimentHandler(exper_mc005_bs, use_logfile=False)
exper_hdl_mc005_bs.set_root_dir(ROOT_DIR)
exper_hdl_mc005_bs.set_model_name("MC-BS-dropout fold-{} (p={})".format(
    exper_hdl_mc005_bs.exper.run_args.fold_ids[0],
    exper_hdl_mc005_bs.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc005_bs.exper.model_name))

exper_mc01 = ExperimentHandler.load_experiment(exp_mc01_model)
exper_hdl_mc01 = ExperimentHandler(exper_mc01, use_logfile=False)
exper_hdl_mc01.set_root_dir(ROOT_DIR)
exper_hdl_mc01.set_model_name("MC-dropout fold-{} (p={})".format(exper_hdl_mc01.exper.run_args.fold_ids[0],
    exper_hdl_mc01.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc01.exper.model_name))

exper_mc01_bs = ExperimentHandler.load_experiment(exp_mc01_bs_model)
exper_hdl_mc01_bs = ExperimentHandler(exper_mc01_bs, use_logfile=False)
exper_hdl_mc01_bs.set_root_dir(ROOT_DIR)
exper_hdl_mc01_bs.set_model_name("MC-BS-dropout fold-{} (p={})".format(
    exper_hdl_mc01_bs.exper.run_args.fold_ids[0],
    exper_hdl_mc01_bs.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc01_bs.exper.model_name))

try:
    del test_set
except:
    pass

test_set = ACDC2017TestHandler(exper_config=exper_mc005.config, 
                               search_mask=config.dflt_image_name + ".mhd", fold_ids=training_fold,
                               debug=False, batch_size=10, use_cuda=True)

  0%|          | 0/10 [00:00<?, ?it/s]

Load from /home/jorg/repository/dcnn_acdc/logs/20180328_10_54_36_dcnn_mcv1_150000E_lr2e02/stats/exper_stats.dll
!!! IMPORTANT !!! - EVALUATING MODELS ON HOLD OUT IMAGES IN FOLD 2
Model name MC-dropout fold-2 (p=0.05)
Load from /home/jorg/repository/dcnn_acdc/logs/20180329_08_04_12_dcnn_mcv1_150000E_lr2e04/stats/exper_stats.dll
Model name MC-BS-dropout fold-2 (p=0.05)
Load from /home/jorg/repository/dcnn_acdc/logs/20180328_10_54_55_dcnn_mcv1_150000E_lr2e02/stats/exper_stats.dll
Model name MC-dropout fold-2 (p=0.1)
Load from /home/jorg/repository/dcnn_acdc/logs/20180329_08_03_29_dcnn_mcv1_150000E_lr2e04/stats/exper_stats.dll
Model name MC-BS-dropout fold-2 (p=0.1)
INFO - File list contains 50 files, hence 25 patients
0 - /home/jorg/repository/dcnn_acdc/data/Folds/fold2/validate/images/patient006_frame01.mhd
1 - /home/jorg/repository/dcnn_acdc/data/Folds/fold2/validate/images/patient006_frame16.mhd


 10%|█         | 1/10 [00:00<00:08,  1.05it/s]

2 - /home/jorg/repository/dcnn_acdc/data/Folds/fold2/validate/images/patient007_frame01.mhd
3 - /home/jorg/repository/dcnn_acdc/data/Folds/fold2/validate/images/patient007_frame07.mhd


 20%|██        | 2/10 [00:01<00:07,  1.06it/s]/home/jorg/anaconda2/envs/pyt_0_3_0/lib/python2.7/site-packages/scipy/ndimage/interpolation.py:616: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


...

16 - /home/jorg/repository/dcnn_acdc/data/Folds/fold2/validate/images/patient029_frame01.mhd
17 - /home/jorg/repository/dcnn_acdc/data/Folds/fold2/validate/images/patient029_frame12.mhd


 90%|█████████ | 9/10 [00:05<00:00,  1.57it/s]

18 - /home/jorg/repository/dcnn_acdc/data/Folds/fold2/validate/images/patient030_frame01.mhd
19 - /home/jorg/repository/dcnn_acdc/data/Folds/fold2/validate/images/patient030_frame12.mhd


100%|██████████| 10/10 [00:06<00:00,  1.59it/s]

INFO - Successfully loaded 10 ED/ES patient pairs


In [6]:
def test_ensemble(test_set, exper_hdl, mc_samples=10, sample_weights=True, use_uncertainty=False,
                  u_threshold=None, image_range=None, verbose=False, reset_results=False, 
                  store_details=False, generate_stats=False, save_results=False, use_seed=False,
                  do_filter=False, checkpoints=None):
    if checkpoints is None:
        checkpoints = [100000, 110000, 120000, 130000, 140000, 150000]  
    if reset_results:
        exper_hdl.reset_results(mc_samples=mc_samples, use_dropout=sample_weights)
    if use_uncertainty:
        print("INFO - Using {:.2f} as uncertainty threshold".format(std_threshold))
    for checkpoint in checkpoints:
        dcnn_model = exper_hdl.load_checkpoint(verbose=False, drop_prob=exper_hdl.exper.run_args.drop_prob, 
                                               checkpoint=checkpoint)
        if image_range is None:
            image_range = np.arange(len(test_set.images))
        num_of_images = len(image_range)
        print("Evaluating model {}".format(exper_hdl.exper.model_name))
        print("INFO - Running test on {} images with model from checkpoint {}".format(num_of_images, checkpoint))
        for image_num in image_range:
            exper_hdl.test(dcnn_model, test_set, image_num=image_num, sample_weights=sample_weights, 
                           mc_samples=mc_samples, compute_hd=True, use_uncertainty=use_uncertainty, 
                           u_threshold=u_threshold, verbose=verbose, store_details=store_details,
                           use_seed=use_seed, do_filter=do_filter)
        del dcnn_model
    exper_hdl.test_results.compute_mean_stats()
    exper_hdl.test_results.show_results()
    
    if generate_stats:
        print("INFO - generating statistics for {} run(s). May take a while".format(exper_hdl.test_results.N))
        exper_hdl.test_results.generate_all_statistics()
    if save_results:
        exper_hdl.test_results.save_results()

# None means all test images will be evaluated
image_range = None

In [8]:
sample_weights = True
mc_samples = 20
store_details = True
image_range = [9]  # [1, 9, 19]
generate_stats = True

save_results = False
#  exper_hdl_mc01 exper_hdl_mc025 exp_hdl_mc01_bs  exper_hdl_mc04  exp_hdl_mc01_bs
exper_handler = exper_hdl_mc005
exper_handler.logger = None

print("INFO - Evaluating model {}".format(exper_handler.exper.model_name))

test_ensemble(test_set, exper_handler, mc_samples=mc_samples, sample_weights=sample_weights,
              use_uncertainty=False, 
              u_threshold=0.05, image_range=image_range, verbose=False, reset_results=True, use_seed=True,
              store_details=store_details, generate_stats=generate_stats, save_results=save_results,
              do_filter=True, checkpoints=[150000])


    
print("Ready")

INFO - Evaluating model MC-dropout fold-2 (p=0.05)
Evaluating model MC-dropout fold-2 (p=0.05)
INFO - Running test on 1 images with model from checkpoint 150000
test_result.use_dropout True
Image 10 - Test accuracy: test loss -0.979	 dice(RV/Myo/LV): ES 0.00/0.90/0.58 --- ED 0.73/0.85/0.86
Image 10 - Test accuracy: test loss -0.979	 Hausdorff(RV/Myo/LV): ES 107.48/16.33/16.33 --- ED 57.21/16.09/17.26
Test accuracy: 	 dice(RV/Myo/LV): ES 0.00 (0.00)/0.90 (0.00)/0.58 (0.00) --- ED 0.73 (0.00)/0.85 (0.00)/0.86 (0.00)
Test accuracy: 	 Hausdorff(RV/Myo/LV): ES 107.48 (0.00)/16.33 (0.00)/16.33 (0.00) --- ED 57.21 (0.00)/16.09 (0.00)/17.26 (0.00)
INFO - generating statistics for 1 run(s). May take a while
Ready


In [9]:
from sklearn import mixture
from matplotlib.colors import LogNorm

In [13]:
def get_uncertainty_maps(testset, image_num, c_phase=0, slice_id=0, uncertainty_type="bald"):
    if uncertainty_type == "bald":
        uncertainty_map = testset.bald_maps[image_num][c_phase, :, :, slice_id]
    else:
        uncertainty_map = testset.uncertainty_maps[image_num][c_phase, :, :, slice_id]
    
    return uncertainty_map        

heat_map = get_uncertainty_maps(exper_handler.test_results, image_num=0, c_phase=1, slice_id=0)
print(heat_map.shape)

un_data = heat_map.ravel()

del clf
gmm = mixture.GaussianMixture(n_components=10, covariance_type='full')
gmm.fit(heat_map)


# plt.imshow(z+10, extent=(np.amin(x), np.amax(x), np.amin(y), np.amax(y)),
#        cmap=cm.hot, norm=LogNorm())

# plt.colorbar()
# plt.show()

(223, 264)


GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
        means_init=None, n_components=10, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [14]:
# display predicted scores by the model as a contour plot
x = np.linspace(0, heat_map.shape[0])
y = np.linspace(0, heat_map.shape[1])
X, Y = np.meshgrid(x, y)
XX = np.array([X.ravel(), Y.ravel()]).T
Z = -clf.score_samples(XX)
Z = Z.reshape(X.shape)
CS = plt.contour(X, Y, Z, norm=LogNorm(vmin=1.0, vmax=1000.0),
                 levels=np.logspace(0, 3, 10))
CB = plt.colorbar(CS, shrink=0.8, extend='both')

plt.title('Negative log-likelihood predicted by a GMM')
plt.axis('tight')
plt.show()

ValueError: Expected the input data X have 264 features, but got 2 features

In [ ]:
clf = mixture.GaussianMixture(n_components=2, covariance_type='full')
clf.fit(X_train)

# display predicted scores by the model as a contour plot
x = np.linspace(-20., 30.)
y = np.linspace(-20., 40.)
X, Y = np.meshgrid(x, y)
XX = np.array([X.ravel(), Y.ravel()]).T
Z = -clf.score_samples(XX)
Z = Z.reshape(X.shape)

CS = plt.contour(X, Y, Z, norm=LogNorm(vmin=1.0, vmax=1000.0),
                 levels=np.logspace(0, 3, 10))
CB = plt.colorbar(CS, shrink=0.8, extend='both')
plt.scatter(X_train[:, 0], X_train[:, 1], .8)

plt.title('Negative log-likelihood predicted by a GMM')
plt.axis('tight')
plt.show()